In [189]:
!pip3 install beautifulsoup4
!pip3 install requests

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [190]:
import requests
from bs4 import BeautifulSoup

# Сбор данных

Данные – отзывы на робот-пылесосы.
Особенность отзывов состоит в том, что они состоят из абзаца достоинст, недостатков и комментария. 
Для формирования списка характерных слов для негативных и положительных отзывов я решила учитывать помимо комментария еще и описание недостатков и достоинств соотвественно. Тем самым увеличила корпус отзывов, так как бывает, что пользователи весь негатив или одобрение помещают в эти разделы. Для тестовых отзывов я учитываю только комментарии.

In [295]:
train_urls = ['https://peredpokupkoy.ru/otzyvy-xrobot-xr-510d/', 
        'https://peredpokupkoy.ru/otzyvy-xrobot-xr-510f/',
        'https://peredpokupkoy.ru/otzyvy-xrobot-strider/',
        'https://peredpokupkoy.ru/otzyvy-xrobot-virage/',
        'https://peredpokupkoy.ru/otzyvy-pullman-pl-1016/']
test_url = 'https://peredpokupkoy.ru/otzyvy-irobot-scooba-450/'

In [192]:
def get_parser(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    comment_content = soup.find_all('div', {'class': 'cm_single_comment_content'})
    return comment_content    

In [193]:
def train_get_neg(comments):
    neg_comments = []
    for comment in comments:
        if comment.find('meta', content='1', itemprop='ratingValue') or comment.find('meta', content='2', itemprop='ratingValue') or comment.find('meta', content='3', itemprop='ratingValue') is not None:
            if comment.find('div', class_='cm_single_comment_limitations_ins') and comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_lim = comment.find('div', class_='cm_single_comment_limitations_ins')
                neg_comments.append(get_lim.text)               
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                neg_comments.append(get_comment.text)
            elif comment.find('div', class_='cm_single_comment_limitations_ins') is not None:
                get_lim = comment.find('div', class_='cm_single_comment_limitations_ins')
                neg_comments.append(get_lim.text) 
            elif comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                neg_comments.append(get_comment.text)                
    return neg_comments

In [194]:
def get_neg(comments):
    neg_comments = []
    for comment in comments:
        if comment.find('meta', content='1', itemprop='ratingValue') or comment.find('meta', content='2', itemprop='ratingValue') or comment.find('meta', content='3', itemprop='ratingValue')  is not None:              
            if comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                neg_comments.append(get_comment.text)
    return neg_comments

In [195]:
def train_get_pos(comments):
    pos_comments = []
    for comment in comments:
        if comment.find('meta', content='4', itemprop='ratingValue') or comment.find('meta', content='5', itemprop='ratingValue') is not None:
            if comment.find('div', class_='cm_single_comment_dignity_ins') and comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_dig = comment.find('div', class_='cm_single_comment_dignity_ins')
                pos_comments.append(get_dig.text)
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                pos_comments.append(get_comment.text)
            elif comment.find('div', class_='cm_single_comment_dignity_ins') is not None:
                get_dig = comment.find('div', class_='cm_single_comment_dignity_ins')
                pos_comments.append(get_dig.text)
            elif comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                pos_comments.append(get_comment.text)                
    return pos_comments

In [196]:
def get_pos(comments):
    pos_comments = []
    for comment in comments:
        if comment.find('meta', content='4', itemprop='ratingValue') or comment.find('meta', content='5', itemprop='ratingValue') is not None:
            if comment.find('div', class_='cm_single_comment_message_ins') is not None:
                get_comment = comment.find('div', class_='cm_single_comment_message_ins')
                pos_comments.append(get_comment.text)
    return pos_comments

In [197]:
train_neg = []
train_pos = []
for url in train_urls:
    comments = get_parser(url)
    neg_comments = train_get_neg(comments)
    train_neg += neg_comments
    pos_comments = train_get_pos(comments)
    train_pos += pos_comments

In [198]:
len(train_neg)

92

In [199]:
len(train_pos)

224

Для дальнейших действий я буду использовать 90 негативных и 90 положительных комментариев.

# Токенизация и начальная форма

In [200]:
!pip3 install nltk
!pip3 install pymystem3

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [201]:
import nltk
import collections
from nltk.tokenize import word_tokenize
from collections import Counter 
from pymystem3 import Mystem
m = Mystem()

Для учитывания частотности характерных слов я сначала взяла из бибилиотеки nltk список стоп-слов для русского языка, а потом начала дополнять этот список словами, котрые оказались частотными для обоих множеств и которые характерны скорее для описания роботов-пылесосов в целом, чем конкретно для положительных или отрицательных отзывов.

In [269]:
sw = [ 'ножка', 'контейнер', 'зарядка', 'время', 'один', 'база', 'весь', 'щетка','работа',
      'день', 'пыль', 'пол', 'дом', 'модель', 'мочь', 'просто', 'очень', 'но', 'убирать',
      'ковер', 'мусор', 'свой', 'волос', 'шерсть', 'работать', 'ну', 'комната', 'говорить',
      'который', 'пылесос', 'это', 'робот', 'квартира', 'и', 'в', 'во', 'что',
      'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'да', 'ты',
      'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'ее', 'мне', 'было', 'вот', 'от', 'меня',
      'еще', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'вдруг', 'ли', 'если', 'уже',
      'или', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'вам', 'ведь', 'там',
      'потом', 'себя', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для',
      'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз',
      'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому',
      'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'мой', 'тем', 'чтобы', 'нее',
      'сейчас', 'были', 'куда', 'зачем', 'при', 'два', 'об', 'другой', 'хоть', 'после',
      'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'них', 'какая',
      'разве', 'три', 'эту', 'моя', 'впрочем', 'свою', 'этой', 'перед', 'иногда', 'чуть',
      'том', 'такой', 'им', 'всегда', 'всю', 'между', 'чистить', 'стоять', 'стул', 'влажный',
      'только','большой','угол','пылесосить', 'место', 'час', 'мебель','уровень','итог',
      'можно', 'стена', 'иметь', 'каждый', 'фильтр', 'много', 'уборка' ]

In [229]:
# создаем частотный словарь, при этом лемматизируем слова, токенизируем и не учитываем стоп-слова
def collect_freqlist(comments, max_len=100):
    freqlist = Counter()
    for text in comments:
        lemmas = m.lemmatize(text)
        norm = ''.join(lemmas)
        words = [w.lower() for w in word_tokenize(norm) if w.isalpha()]
        filtered = [w for w in words if w not in sw]
        for word in filtered:
            freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

In [106]:
# находим общие для двух множеств частотные слова, чтобы добавить в стоп-слова
same_keys = []
for key in collect_freqlist(train_neg).keys():
    for k in collect_freqlist(train_pos).keys():
        if key == k:
            same_keys.append(key)          

В процессе составления характерных слов получилось, что *не* – самое частотное слово и для положительных, и для негативных отзывов, *хороший* и *хорошо* тоже встретились в обоих списках. Для повышения качества определения, я оставила *не* только в списке негативных отзывов, а *хороший*, *хорошо* – только в положительном.

In [313]:
clean_pos = collect_freqlist(train_pos[0:90])

In [314]:
clean_pos.pop('не')

131

In [316]:
clean_neg = collect_freqlist(train_neg[0:90])

In [317]:
clean_neg.pop('хороший')
clean_neg.pop('хорошо')

14

# Тональность

In [325]:
import collections
from collections import Counter

In [326]:
freq_lists = {}
freq_lists['negative'] = clean_neg
freq_lists['positive'] = clean_pos

In [327]:
def simple_snt_detect(freq_lists, text):
    counts = Counter()
    for snt, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(' '.join(m.lemmatize(txt))):
            counts[snt] += int(freq_list[word] > 0)
    return counts.most_common()

# Оценка

In [210]:
!pip3 install sklearn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [211]:
from sklearn.metrics import accuracy_score

In [310]:
sentiment = ('negative', 'positive')

def get_comment_for_snt(snt, page):
    if snt == 'negative':
        final = get_neg(page)
    elif snt == 'positive':
        final = get_pos(page)
    return final[0:5]    

In [328]:
def test_simple_snt_detect(freq_lists, page):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходную тональность
    for snt in sentiment:
        for text in get_comment_for_snt(snt, page):
            predicted_snt = simple_snt_detect(freq_lists, text)[0][0]
            results.append(predicted_snt)
            gold.append(snt)
    print("RESULTS:")
    print("%d sentiments" % len(sentiment))
    print("Accuracy: %.4f" % accuracy_score(results, gold))  

In [329]:
test_simple_snt_detect(freq_lists, get_parser(test_url))

RESULTS:
2 sentiments
Accuracy: 0.5000
